The following code is using Reddit API to contact the moderators of subreddits. However the API does not work for private subreddits ()

Many subreddits have gone private this year (summer 2023) in protest of Reddit adding charges to the commercial use of their API (https://mobilewirelesstrends.com/apps/why-are-subreddits-going-private/). This might be a reason for why the subreddits in my list have been set to private. 

The only way to contact the moderators of private subreddits, is via the url (for ex: https://www.reddit.com//r/asd/). This can only be done in the browser. This will lead to a page where it is stated that the subreddit ( ex. r/asd ) is a private community, and an option to contact the moderators is available. If this button ( contact the moderators ) is clicked, the user is sent to a new page/url where there is a form field with a subject input and a message input ( up to this point the process of accessing the subreddit's page, clicking on contact the moderators and filling in the subject and message in the form , can be automated using Python and  Selenium ). After these, there is a Captcha tick box ( this cannot be automated ) and after this Captcha has been clicked and confirmed user is not a bot, the message can be sent. 

I will thus automate the first part of the process for all the private subreddits ( using Selenium - https://www.selenium.dev/documentation/). After the message is filled in, I will need to manually click the Captcha and send the message.  - see script in auto_reddit_mod_contact_selenium.ipynb

In [2]:
import os
import praw
from dotenv import load_dotenv


In [3]:
load_dotenv()
# API credentials
CLIENT_SECRET = os.environ.get('CLIENT_SECRET')
CLIENT_ID = os.environ.get('CLIENT_ID')
REDDIT_USERNAME = os.getenv('REDDIT_USERNAME')
REDDIT_PASSWORD = os.getenv('REDDIT_PASSWORD')


In [4]:
# get access to the Reddit API
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    username=REDDIT_USERNAME,
    password=REDDIT_PASSWORD,
    user_agent="MyMsciAPI/0.0.1",   # this can be named anything
)


In [15]:
# Get all private subreddits
import pandas as pd

asd_private_subreddits = pd.read_csv('asd_new_null_rows_list.csv')
asd_private_subreddits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             36 non-null     int64  
 1   restrict_posting       0 non-null      float64
 2   display_name           36 non-null     object 
 3   title                  36 non-null     object 
 4   display_name_prefixed  36 non-null     object 
 5   subscribers            0 non-null      float64
 6   name                   36 non-null     object 
 7   public_description     26 non-null     object 
 8   community_reviewed     0 non-null      float64
 9   created                36 non-null     float64
 10  subreddit_type         36 non-null     object 
 11  id                     36 non-null     object 
 12  over18                 0 non-null      float64
 13  header_title           0 non-null      float64
 14  description            0 non-null      float64
 15  url     

In [35]:
asd_private_subreddits[['display_name', 'url']]

,display_name,url
0,neurodivergents_ZA,/r/neurodivergents_ZA/
1,LateStageAutism,/r/LateStageAutism/
2,asd,/r/asd/
3,AutisticWomen,/r/AutisticWomen/
4,HighFunctioning,/r/HighFunctioning/
5,AutismArtistic,/r/AutismArtistic/
6,autismspectrum,/r/autismspectrum/
7,AspiePartners,/r/AspiePartners/
8,autismus_treff,/r/autismus_treff/
9,bottomlulz,/r/bottomlulz/


In [32]:
# Send a message to moderators
message_subject = 'Request access for academic research'
# Get the message
with open("private_subreddits_contact_letter.txt","r") as file:
    message_text=file.read()
message_text

"Hello, \n\nMy name is Andreea, and I am a data science master's student at Birkbeck College, University of London. My thesis research focuses on prevalent discussion themes within autism Reddit communities. In particular, I'm planning to track the frequency of specific keywords linked to Applied Behavioural Analysis and assess the overall sentiment of discussions revolving around this topic. My aim is to gain a comprehensive understanding of how the autism community perceives Applied Behavioural Analysis. I am reaching out to respectfully request permission to join and access the discussions in your private subreddit. \n\nI understand how crucial privacy and security are in your community, and thus I am committed to maintaining the anonymity and confidentiality of all members. The information gathered will be used strictly for academic purposes. \n\nIn recognition of the trust and sensitivity associated with your community's discussions, I am more than willing to adhere to any guideli

In [25]:
def contact_mod(subreddit_name):
    subreddit= reddit.subreddit(subreddit_name)
    moderator_usernames = [mod.name for mod in subreddit.moderator()]
    print(moderator_usernames)
    subreddit.message(subject=message_subject, message=message_text)
    print('Message sent successfully to moderators.')

In [31]:
for sub in asd_private_subreddits['display_name']:
    try:
        contact_mod(sub)
    except Exception:
        print(sub, 'cannot be contacted via API due to privacy')


neurodivergents_ZA cannot be contacted via API due to privacy
LateStageAutism cannot be contacted via API due to privacy
asd cannot be contacted via API due to privacy
AutisticWomen cannot be contacted via API due to privacy
HighFunctioning cannot be contacted via API due to privacy
AutismArtistic cannot be contacted via API due to privacy
autismspectrum cannot be contacted via API due to privacy
AspiePartners cannot be contacted via API due to privacy
autismus_treff cannot be contacted via API due to privacy
bottomlulz cannot be contacted via API due to privacy
DiagnoseThem cannot be contacted via API due to privacy
HighFunctioningAutism cannot be contacted via API due to privacy
autismacceptance cannot be contacted via API due to privacy
Autism_Awareness cannot be contacted via API due to privacy
AspieGirlsDND cannot be contacted via API due to privacy
hydewars cannot be contacted via API due to privacy
AnchorageCovid19 cannot be contacted via API due to privacy
Spectrum_Gaming canno